In [6]:
# Import modules: xrray, schemaorg
import xarray
from schemaorg.main import Schema 
from schemaorg.main.parse import RecipeParser                # For schema validation
from schemaorg.templates.google import make_dataset          # To generate html


# Load data with metadata (Type: netcdf)
data = xarray.open_dataset('data/ampere/20130101.1300.3600.600.north.grd.ncdf')
data

<xarray.Dataset>
Dimensions:    (block: 6, vector: 1200)
Dimensions without coordinates: block, vector
Data variables: (12/29)
    nlon       (block) int32 ...
    nlat       (block) int32 ...
    start_yr   (block) int32 ...
    start_mo   (block) int32 ...
    start_dy   (block) int32 ...
    start_hr   (block) int32 ...
    ...         ...
    ddBeast2   (block, vector) float32 ...
    dBr        (block, vector) float32 ...
    ddBr       (block, vector) float32 ...
    Jr         (block, vector) float32 ...
    dJr        (block, vector) float32 ...
    dBprt      (block, vector) float32 ...

In [7]:
## Create a 'Dataset' Schema object
SchemaObj = Schema("Dataset")

## Add a single property
SchemaObj.add_property("name","AMPERE Data For 01/01/2013 13:00:00")

SchemaObj.add_property("description ","AMPERE Data")
SchemaObj.add_property("id","AMPERE Data For 01/01/2013 13:00:00")

Specification base set to https://www.schema.org
Using Version 12.0
Found https://www.schema.org/Dataset
Dataset: found 129 properties


In [13]:
from datetime import datetime

start_yr = data.data_vars['start_yr'][0]
start_mo = data.data_vars['start_mo'][0]
start_dy = data.data_vars['start_dy'][0]
start_hr = data.data_vars['start_hr'][0]
start_min = data.data_vars['start_mt'][0]
start_sec = data.data_vars['start_sc'][0]


end_yr = data.data_vars['end_yr'][-1]
end_mo = data.data_vars['end_mo'][-1]
end_dy = data.data_vars['end_dy'][-1]
end_hr = data.data_vars['end_hr'][-1]
end_min = data.data_vars['end_mt'][-1]
end_sec = data.data_vars['end_sc'][-1]

start_dt = datetime(start_yr,start_mo,start_dy,start_hr,start_min,start_sec)

end_dt = datetime(end_yr,end_mo,end_dy,end_hr,end_min,end_sec)

SchemaObj.add_property('temporalCoverage',f'{start_dt.astimezone().isoformat()}/{end_dt.astimezone().isoformat()}')


In [9]:
# Variables Measured

variable_measured_list = []

prop1 = Schema("PropertyValue")
prop1.add_property('name',data.data_vars['dBnorth1'].attrs['description'])
prop1.add_property('description',data.data_vars['dBnorth1'].attrs['description'])
prop1.add_property('unitText','nT')

variable_measured_list.append(prop1)

prop2 = Schema("PropertyValue")
prop2.add_property('name',data.data_vars['dBnorth2'].attrs['description'])
prop2.add_property('description',data.data_vars['dBnorth2'].attrs['description'])
prop2.add_property('unitText','nT')

variable_measured_list.append(prop2)
SchemaObj.add_property('variableMeasured',variable_measured_list);




Specification base set to https://www.schema.org
Using Version 12.0
Found https://www.schema.org/PropertyValue
PropertyValue: found 20 properties
Specification base set to https://www.schema.org
Using Version 12.0
Found https://www.schema.org/PropertyValue
PropertyValue: found 20 properties


In [14]:
# To generate only the json markup
json_markup = make_dataset(SchemaObj,template="google/dataset.html")
## Context should be https://schema.org
json_markup = json_markup.replace('"@context": "https://www.schema.org"','"@context": "https://schema.org/"')


# To generate HTML landing page with json markup embedded
html = make_dataset(SchemaObj,template="google/dataset-vue-table.html")
html = html.replace('"@context": "https://www.schema.org"','"@context": "https://schema.org/"')

print(json_markup)

<script type="application/ld+json">
{
    "name": "AMPERE Data For 01/01/2013 13:00:00",
    "temporalCoverage": "2013-01-01T13:00:00+00:00/2013-01-01T14:00:00+00:00",
    "variableMeasured": [
        {
            "name": "Magnetic perturbation parallel to the AACGM northward direction [nT]",
            "description": "Magnetic perturbation parallel to the AACGM northward direction [nT]",
            "unitText": "nT",
            "@type": "PropertyValue"
        },
        {
            "name": "Magnetic perturbation perpendicular to the AACGM eastward direction [nT]",
            "description": "Magnetic perturbation perpendicular to the AACGM eastward direction [nT]",
            "unitText": "nT",
            "@type": "PropertyValue"
        }
    ],
    "@context": "https://schema.org/",
    "@type": "Dataset"
}
</script>

